In [12]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import r2_score 

from alpaca.ue import MCDUE
from alpaca.utils.datasets.builder import build_dataset
from alpaca.utils.ue_metrics import ndcg
from alpaca.ue.masks import BasicBernoulliMask
import alpaca.nn as ann

In [2]:
# Load dataset
dataset = build_dataset('kin8nm', val_split=1_000)
x_train, y_train = dataset.dataset('train')
x_val, y_val = dataset.dataset('val')
x_train.shape, y_val.shape
train_ds = TensorDataset(torch.DoubleTensor(x_train), torch.DoubleTensor(y_train))
val_ds = TensorDataset(torch.DoubleTensor(x_val), torch.DoubleTensor(y_val))
train_loader = DataLoader(train_ds, batch_size=512)
val_loader = DataLoader(val_ds, batch_size=512)

In [5]:
class BaseMLP(nn.Module):
    def __init__(self, layer_sizes, activation, postprocessing=lambda x: x, device=None):
        super(BaseMLP, self).__init__()

        if device is None:
            self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        else:
            self.device = device

        self.layer_sizes = layer_sizes
        self.fcs = []
        for i, layer in enumerate(layer_sizes[:-1]):
            fc = nn.Linear(layer, layer_sizes[i+1])
            setattr(self, 'fc'+str(i), fc)  # to register params
            self.fcs.append(fc)
        self.postprocessing = postprocessing
        self.activation = activation
        self.double()
        self.to(self.device)

    def forward(self, x, dropout_rate=0, dropout_mask=None):
        x = self.activation(self.fcs[0](x))

        for layer_num, fc in enumerate(self.fcs[1:-1]):
            x = self.activation(fc(x))
            if dropout_mask is None:
                x = nn.Dropout(dropout_rate)(x)
            else:
                x = x*dropout_mask(x, dropout_rate, layer_num).double()
        x = self.fcs[-1](x)
        x = self.postprocessing(x)
        return x

class MLP(BaseMLP):
    def __init__(self, layer_sizes, postprocessing=None, loss=nn.MSELoss,
                 optimizer=None, activation=None, **kwargs):
        if postprocessing is None:
            postprocessing = lambda x: x

        if activation is None:
            activation = F.celu

        super(MLP, self).__init__(layer_sizes, activation=activation, postprocessing=postprocessing, **kwargs)

        self.criterion = loss()

        if optimizer is None:
            optimizer = {'type': 'Adadelta'}
        self.optimizer = self.init_optimizer(optimizer)

    def init_optimizer(self, optimizer):
        if isinstance(optimizer, dict):
            kwargs = optimizer.copy()
            opt_type = getattr(torch.optim, kwargs.pop('type'))
            kwargs['params'] = self.parameters()
            optimizer = opt_type(**kwargs)
        elif not isinstance(optimizer, torch.optim.Optimizer):
            raise TypeError(
                'optimizer must be either an Optimizer object or a dict, '
                'but got {}'.format(type(optimizer)))
        return optimizer

In [7]:
# Train model
layers = (8, 256, 128, 64, 1)
model = MLP(layers)

In [9]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

model.train()
for epochs in range(10):
    for x_batch, y_batch in train_loader: # Train for one epoch
        predictions = model(x_batch)
        loss = criterion(predictions, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
print('Train loss on last batch', loss.item())

Train loss on last batch 0.015673447147550233


In [13]:
# Check model effectiveness 
model.eval()
x_batch, y_batch = next(iter(val_loader))
predictions = model(x_batch).detach().cpu().numpy()
print('R2', r2_score(predictions, y_batch))

R2 0.5611923085696566


In [16]:
# Calculate uncertainty estimation
estimator = MCDUE(model)
predictions, estimations = estimator.estimate(x_batch)

Uncertainty estimation with MCDUE_regression approach: 100%|██████████| 25/25 [00:00<00:00, 169.74it/s]


In [19]:
# Calculate NDCG score for the uncertainty
errors = np.abs(estimations - y_batch.reshape((-1)).numpy()) 
score = ndcg(np.array(errors), estimations)
print("Quality score is ", score)

Quality score is  0.9254815020030868
